# Univariate LSTM model for volatility forecasting

## 1. Importing packages 

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import t
import collections

2023-06-08 15:22:34.867067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1.1 Importing functions  

In [3]:
from Model_functions import scaled_data, make_supervised, test_batch, lstm_model, generate_2_forecasts, generate_2_forecasts_hybrid, generate_forecasts_hybrid_no_rf
from Model_functions import calc_errors, calc_errors_ms, univariate_lstm_preds, dm_test_2, calc_errors_lstm, rv_calc
from Model_functions import make_supervised_shift, univariate_lstm_preds_shift, generate_2_forecasts_hybrid_shift, create_h_step_forecasts, produce_h_step_forecasting_errors

## 2. Unpacking simulated sequences 

In [4]:
sim_arch_draws = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_arch_draws.csv')
sim_arch_condvol = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_arch_condvol.csv')
sim_garch_draws = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_garch_draws.csv')
sim_garch_condvol = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_garch_condvol.csv')
sim_egarch_draws = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_egarch_draws.csv')
sim_egarch_condvol = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_egarch_condvol.csv')
sim_gjrgarch_draws = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_gjrgarch_draws.csv')
sim_gjrgarch_condvol = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_gjrgarch_condvol.csv')
sim_msgarch_draws = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_msgarch_draws.csv')
sim_msgarch_condvol = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Simulated_sequences/sim_msgarch_condvol.csv')

## 2.1 Unpacking features of the simulated sequences (distr = "std")

In [5]:
p = 5 # Amount of models

### 2.1.1 ARCH sequence

In [6]:
# 1-step ahead features
arch_1_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_1_ARCH.csv')
arch_1_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_1_GARCH.csv')
arch_1_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_1_EGARCH.csv')
arch_1_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_1_GJRGARCH.csv')
arch_1_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_1_MSGARCH.csv')

# MS-GARCH feature is longer
diff = len(arch_1_msgarch) - len(arch_1_gjrgarch)

pred_matrix_arch_1 = np.hstack((arch_1_arch, arch_1_garch, arch_1_egarch, arch_1_gjrgarch, arch_1_msgarch[diff:]))

# 12-step ahead features
arch_12_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_12_ARCH.csv')
arch_12_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_12_GARCH.csv')
arch_12_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_12_EGARCH.csv')
arch_12_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_12_GJRGARCH.csv')
arch_12_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_12_MSGARCH.csv')
pred_matrix_arch_12 = np.hstack((arch_12_arch, arch_12_garch, arch_12_egarch, arch_12_gjrgarch, arch_12_msgarch))

# 24-step ahead features
arch_24_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_24_ARCH.csv')
arch_24_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_24_GARCH.csv')
arch_24_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_24_EGARCH.csv')
arch_24_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_24_GJRGARCH.csv')
arch_24_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/ARCH sequence/ARCH_24_MSGARCH.csv')
pred_matrix_arch_24 = np.hstack((arch_24_arch, arch_24_garch, arch_24_egarch, arch_24_gjrgarch, arch_24_msgarch))

### 2.1.2 GARCH sequence

In [7]:
# 1-step ahead features
garch_1_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_1_ARCH.csv')
garch_1_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_1_GARCH.csv')
garch_1_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_1_EGARCH.csv')
garch_1_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_1_GJRGARCH.csv')
garch_1_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_1_MSGARCH.csv')

# MS-GARCH feature is longer
diff = len(garch_1_msgarch) - len(garch_1_gjrgarch)

pred_matrix_garch_1 = np.hstack((garch_1_arch, garch_1_garch, garch_1_egarch, garch_1_gjrgarch, garch_1_msgarch[diff:]))

# 12-step ahead features
garch_12_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_12_ARCH.csv')
garch_12_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_12_GARCH.csv')
garch_12_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_12_EGARCH.csv')
garch_12_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_12_GJRGARCH.csv')
garch_12_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_12_MSGARCH.csv')
pred_matrix_garch_12 = np.hstack((garch_12_arch, garch_12_garch, garch_12_egarch, garch_12_gjrgarch, garch_12_msgarch))

# 24-step ahead features
garch_24_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_24_ARCH.csv')
garch_24_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_24_GARCH.csv')
garch_24_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_24_EGARCH.csv')
garch_24_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_24_GJRGARCH.csv')
garch_24_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GARCH sequence/GARCH_24_MSGARCH.csv')
pred_matrix_garch_24 = np.hstack((garch_24_arch, garch_24_garch, garch_24_egarch, garch_24_gjrgarch, garch_24_msgarch))

### 2.1.3 E-GARCH sequence 

In [8]:
# 1-step ahead features
egarch_1_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_1_ARCH.csv')
egarch_1_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_1_GARCH.csv')
egarch_1_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_1_EGARCH.csv')
egarch_1_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_1_GJRGARCH.csv')
egarch_1_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_1_MSGARCH.csv')

# MS-GARCH feature is longer
diff = len(egarch_1_msgarch) - len(egarch_1_gjrgarch)

pred_matrix_egarch_1 = np.hstack((egarch_1_arch, egarch_1_garch, egarch_1_egarch, egarch_1_gjrgarch, egarch_1_msgarch[diff:]))

# 12-step ahead features
egarch_12_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_12_ARCH.csv')
egarch_12_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_12_GARCH.csv')
egarch_12_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_12_EGARCH.csv')
egarch_12_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_12_GJRGARCH.csv')
egarch_12_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_12_MSGARCH.csv')
pred_matrix_egarch_12 = np.hstack((egarch_12_arch, egarch_12_garch, egarch_12_egarch, egarch_12_gjrgarch, egarch_12_msgarch))

# 24-step ahead features
egarch_24_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_24_ARCH.csv')
egarch_24_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_24_GARCH.csv')
egarch_24_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_24_EGARCH.csv')
egarch_24_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_24_GJRGARCH.csv')
egarch_24_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/E-GARCH sequence/EGARCH_24_MSGARCH.csv')
pred_matrix_egarch_24 = np.hstack((egarch_24_arch, egarch_24_garch, egarch_24_egarch, egarch_24_gjrgarch, egarch_24_msgarch))

### 2.1.4 GJR-GARCH sequence

In [9]:
# 1-step ahead features
gjrgarch_1_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_1_ARCH.csv')
gjrgarch_1_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_1_GARCH.csv')
gjrgarch_1_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_1_EGARCH.csv')
gjrgarch_1_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_1_GJRGARCH.csv')
gjrgarch_1_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_1_MSGARCH.csv')

# MS-GARCH feature is longer
diff = len(gjrgarch_1_msgarch) - len(gjrgarch_1_gjrgarch)

pred_matrix_egarch_1 = np.hstack((gjrgarch_1_arch, gjrgarch_1_garch, gjrgarch_1_egarch, gjrgarch_1_gjrgarch, gjrgarch_1_msgarch[diff:]))

# 12-step ahead features
gjrgarch_12_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_12_ARCH.csv')
gjrgarch_12_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_12_GARCH.csv')
gjrgarch_12_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_12_EGARCH.csv')
gjrgarch_12_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_12_GJRGARCH.csv')
gjrgarch_12_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_12_MSGARCH.csv')
pred_matrix_gjrgarch_12 = np.hstack((gjrgarch_12_arch, gjrgarch_12_garch, gjrgarch_12_egarch, gjrgarch_12_gjrgarch, gjrgarch_12_msgarch))

# 24-step ahead features
gjrgarch_24_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_24_ARCH.csv')
gjrgarch_24_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_24_GARCH.csv')
gjrgarch_24_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_24_EGARCH.csv')
gjrgarch_24_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_24_GJRGARCH.csv')
gjrgarch_24_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/GJR-GARCH sequence/GJRGARCH_24_MSGARCH.csv')
pred_matrix_gjrgarch_24 = np.hstack((gjrgarch_24_arch, gjrgarch_24_garch, gjrgarch_24_egarch, gjrgarch_24_gjrgarch, gjrgarch_24_msgarch))

### 2.1.5 MS-GARCH sequence

In [10]:
# 1-step ahead features
msgarch_1_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_ARCH.csv')
msgarch_1_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GARCH.csv')
msgarch_1_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_EGARCH.csv')
msgarch_1_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GJRGARCH.csv')
msgarch_1_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_MSGARCH.csv')

# MS-GARCH feature is longer
diff = len(msgarch_1_msgarch) - len(msgarch_1_gjrgarch)

pred_matrix_egarch_1 = np.hstack((msgarch_1_arch, msgarch_1_garch, msgarch_1_egarch, msgarch_1_gjrgarch, msgarch_1_msgarch[diff:]))

# 12-step ahead features
msgarch_12_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_ARCH.csv')
msgarch_12_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GARCH.csv')
msgarch_12_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_EGARCH.csv')
msgarch_12_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GJRGARCH.csv')
msgarch_12_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_MSGARCH.csv')
pred_matrix_msgarch_12 = np.hstack((msgarch_12_arch, msgarch_12_garch, msgarch_12_egarch, msgarch_12_gjrgarch, msgarch_12_msgarch[diff:]))

# 24-step ahead features
msgarch_24_arch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_ARCH.csv')
msgarch_24_garch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GARCH.csv')
msgarch_24_egarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_EGARCH.csv')
msgarch_24_gjrgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_GJRGARCH.csv')
msgarch_24_msgarch = pd.read_csv('4. GARCH Modelling/2_Simulated_sequences/Features_simulations/MS-GARCH sequence/MSGARCH_1_MSGARCH.csv')
pred_matrix_msgarch_24 = np.hstack((msgarch_24_arch, msgarch_24_garch, msgarch_24_egarch, msgarch_24_gjrgarch, msgarch_24_msgarch[diff:]))

# 4. LSTM model 

## 4.1 ARCH data

In [11]:
# Calculate realized volatility based on simulated log-returns
p = 50 # Window size
rv_arch = rv_calc(sim_arch_draws, p)

# Realized volatility data
scaled_rv_arch = scaled_data(rv_arch)
scaled_rv_arch = scaled_rv_arch.flatten()

# Conditional volatility benchmark
benchmark_vol = sim_arch_condvol

In [14]:
# Realized volatilit benchmark
benchmark_rv = rv_arch[(len(rv_arch)-20000):]

## 4.1.1 ARCH sequence: 1-step ahead forecasting

In [16]:
# Model parameters
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [18]:
# 1-step ahead forecasts
h = 1
prediction_matrix_lstm_a1 = produce_h_step_forecasting_errors(scaled_rv_arch, pred_matrix_arch_1, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift, benchmark_rv)

313/313 [==============================] - 5s 16ms/step
MSE: LSTM & hybrid models 1-step ahead forecast
2.1731045108145035e-06
2.857102220552215e-06
2.2544931799405955e-06
2.059830483795419e-06
2.604219021157532e-06
2.3317375328802166e-06
MAE: LSTM & hybrid models 1-step ahead forecast
0.0007348593348332858
0.0010328636772684893
0.0007604019667966373
0.0005724949228745718
0.000850688603302902
0.0006796434443191216
MSE & MAE: realized volatility 1-step ahead forecast
1.8834623837809293e-06
0.0003125666128975547


## 4.1.2 ARCH sequence: 12-step ahead forecasting

In [20]:
# Model parameters
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# 12-step ahead forecasts
h = 12
prediction_matrix_lstm_a12 = produce_h_step_forecasting_errors(scaled_rv_arch, pred_matrix_arch_12, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift, benchmark_vol)

313/313 [==============================] - 42s 127ms/step
MSE: LSTM & hybrid models 12-step ahead forecast
0.00025270670090795966
0.0002554637581581183
0.0002500932752121629
0.0002515771630363898
0.0002501456077773355
0.00025345065394293036
MAE: LSTM & hybrid models 12-step ahead forecast
0.007595095089215945
0.007757021588580639
0.007522505431328606
0.007700473982975502
0.00738395233415878
0.00748932453968607
MSE & MAE: realized volatility 12-step ahead forecast
0.0002801909544288314
0.008297771760459234


In [21]:
print(np.shape(prediction_matrix_lstm_a12))

(120024,)


## 4.1.3 24-step ahead forecasts 

In [22]:
# Model parameters
obs_test = 20000
start = 10000
d = 300
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# 24-step ahead forecasts
h = 24
prediction_matrix_lstm_a24 = produce_h_step_forecasting_errors(scaled_rv_arch, pred_matrix_arch_24, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift, benchmark_vol)

313/313 [==============================] - 33s 101ms/step
MSE: LSTM & hybrid models 24-step ahead forecast
0.00024769141498975363
0.00023329626348757526
0.0002488821852310164
0.0002439877815385745
0.00024792138217890914
0.00025535359397194015
MAE: LSTM & hybrid models 24-step ahead forecast
0.007557551148222523
0.006836918930361233
0.007732080872464542
0.0070395205618879
0.007276331922199134
0.00818344925602692
MSE & MAE: realized volatility 24-step ahead forecast
0.0002807182026868349
0.00831158842685687


## 4.2 GARCH sequence

In [21]:
# Calculate realized volatility based on simulated log-returns
p = 50
rv_garch = rv_calc(sim_garch_draws, p)

# Realized volatility data
scaled_rv_garch = scaled_data(rv_garch)
scaled_rv_garch = scaled_rv_garch.flatten()

# Conditional volatility benchmark
benchmark_vol = sim_garch_condvol

## 4.2.1 1-step ahead forecasts

In [22]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 1
prediction_matrix_lstm_g1 = produce_h_step_forecasting_errors(scaled_rv_garch, pred_matrix_garch_1, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift, benchmark_rv)

313/313 [==============================] - 5s 15ms/step
MSE: LSTM & hybrid models 1-step ahead forecast
0.0011943873396769822
0.0011419301273255507
0.001125374610548383
0.0011263174226431522
0.0011330354681243014
0.0011174387452976729
MAE: LSTM & hybrid models 1-step ahead forecast
0.026334354622384256
0.025510707928953543
0.025324541297091416
0.02524181744480277
0.025504268545355452
0.025172085530821987
MSE & MAE: realized volatility 1-step ahead forecast
0.00118345876034133
0.025929801609727382


## 4.2.2 12-step ahead forecasts 

In [26]:
obs_test = 20000
start = 10000
d = 100
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 12-step ahead forecasts using LSTM
h = 12
prediction_matrix_lstm_g12 = produce_h_step_forecasting_errors(scaled_rv_garch, pred_matrix_garch_12, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift, benchmark_vol)

313/313 [==============================] - 11s 33ms/step
MSE: LSTM & hybrid models 12-step ahead forecast
0.0002270329039443998
0.0002229489280347488
0.00024249458396723652
0.0002585782550063766
0.00023947720249999058
0.00022051829377371876
MAE: LSTM & hybrid models 12-step ahead forecast
0.008782544494855918
0.008906249618953019
0.009543768537254487
0.009962986742340578
0.009324533573805546
0.009487866918108873
MSE & MAE: realized volatility 12-step ahead forecast
0.0004005976895277957
0.011695176120330377


## 4.2.3 24-step ahead forecasts

In [27]:
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 24-step ahead forecasts using LSTM
h = 24
prediction_matrix_lstm_g24 = produce_h_step_forecasting_errors(scaled_rv_garch, pred_matrix_garch_24, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift, benchmark_vol)

313/313 [==============================] - 27s 82ms/step
MSE: LSTM & hybrid models 24-step ahead forecast
0.0002734446915533655
0.0002663576756569243
0.00028961382492394185
0.00027707132223657997
0.00028159924433013674
0.00027922802625811246
MAE: LSTM & hybrid models 24-step ahead forecast
0.009846336800413214
0.00966418269245034
0.010129889045876752
0.009729092464440836
0.00991066619487484
0.00981576124539992
MSE & MAE: realized volatility 24-step ahead forecast
0.0005472673261666776
0.013473071454425724


# 4.3 E-GARCH

In [24]:
# Calculate realized volatility based on simulated log-returns
p = 50
rv_egarch = rv_calc(sim_egarch_draws, p)

# Realized volatility data
scaled_rv_egarch = scaled_data(rv_egarch)
scaled_rv_egarch = scaled_rv_egarch.flatten()

# Conditional volatility benchmark
benchmark_vol = sim_egarch_condvol

## 4.3.1 1-step ahead forecasts

In [25]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 1
prediction_matrix_lstm_e1 = produce_h_step_forecasting_errors(scaled_rv_egarch, pred_matrix_garch_1, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift, benchmark_rv)

313/313 [==============================] - 5s 14ms/step
MSE: LSTM & hybrid models 1-step ahead forecast
0.9665350931011987
0.9592536255136719
0.9573564907269486
0.9586057596913747
0.971240569158763
0.9708108866338879
MAE: LSTM & hybrid models 1-step ahead forecast
0.976359285469525
0.972467919376429
0.9716456718323511
0.9721649596403147
0.9785366566688165
0.9781052538977162
MSE & MAE: realized volatility 1-step ahead forecast
0.9635448711035295
0.9744831479149083


## 4.3.2 12-step ahead forecasts

In [31]:
obs_test = 20000
start = 10000
d = 100
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 12
prediction_matrix_lstm_e12 = produce_h_step_forecasting_errors(scaled_rv_egarch, pred_matrix_garch_12, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift, benchmark_vol)

313/313 [==============================] - 13s 42ms/step
MSE: LSTM & hybrid models 12-step ahead forecast
0.013305858890388213
0.01264515921418612
0.012815785743780155
0.012680008154156165
0.012836522230022428
0.012409843645260201
MAE: LSTM & hybrid models 12-step ahead forecast
0.09302435131590822
0.09045063108586403
0.0910156213354964
0.09013237516673278
0.09118071891517733
0.08981366284802669
MSE & MAE: realized volatility 12-step ahead forecast
0.015104236426558474
0.09859234937300014


## 4.3.3 24-step ahead forecasts

In [32]:
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 24-step ahead forecasts using LSTM
h = 24
prediction_matrix_lstm_e24 = produce_h_step_forecasting_errors(scaled_rv_egarch, pred_matrix_garch_24, start, obs_test, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift, benchmark_vol)

157/157 [==============================] - 12s 73ms/step
MSE: LSTM & hybrid models 24-step ahead forecast
0.009863435287439314
0.010304975485335445
0.009564446257292298
0.010862404925743288
0.010569135858370323
0.010086054304371204
MAE: LSTM & hybrid models 24-step ahead forecast
0.07903831764818121
0.08008399311020321
0.07697044865519997
0.08362864433582258
0.08165217323547498
0.07952579065370959
MSE & MAE: realized volatility 24-step ahead forecast
0.013898972476662738
0.09376322662966977


# 4.4 GJR-GARCH data 

In [186]:
# Calculate realized volatility based on simulated log-returns
p = 50
rv_gjrgarch = rv_calc(sim_gjrgarch_draws, p)

# Realized volatility data
scaled_rv_gjrgarch = scaled_data(rv_gjrgarch)
scaled_rv_gjrgarch = scaled_rv_gjrgarch.flatten()

# Conditional volatility benchmark
benchmark_vol = sim_gjrgarch_condvol

## 4.4.1 1-step ahead forecasts

In [187]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 1
prediction_matrix_lstm_r1 = produce_h_step_forecasting_errors(scaled_rv_gjrgarch, pred_matrix_gjrgarch_1, start, obs_test, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift, benchmark_vol)

313/313 [==============================] - 7s 21ms/step
MSE: LSTM & hybrid models 1-step ahead forecast
0.0008510110052683864
0.000766680078264896
0.0008940522017543964
0.0009747299437629154
0.0007004051473597653
0.000695378528647613
MAE: LSTM & hybrid models 1-step ahead forecast
0.015456708346578578
0.014507898759430949
0.01896919289035249
0.020461791459352355
0.012789555353578798
0.01388827847680162
MSE & MAE: realized volatility 1-step ahead forecast
0.0007906618097120884
0.014420564284997571


## 4.4.2 12-step ahead forecasts 

In [188]:
obs_test = 20000
start = 10000
d = 100
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 12
prediction_matrix_lstm_r12 = produce_h_step_forecasting_errors(scaled_rv_gjrgarch, pred_matrix_gjrgarch_12, start, obs_test, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift, benchmark_vol)

313/313 [==============================] - 13s 41ms/step
MSE: LSTM & hybrid models 12-step ahead forecast
0.0011389952238127865
0.001157873451795153
0.0008432931059182566
0.0008560720296687262
0.0009820425484266828
0.0008730136181233633
MAE: LSTM & hybrid models 12-step ahead forecast
0.019458586583858684
0.022764029069583882
0.015171567991933255
0.016369039320354942
0.020202760631373946
0.014892220037731118
MSE & MAE: realized volatility 12-step ahead forecast
0.0012777964770205295
0.017995410326928647


## 4.4.3 24-step ahead forecasts

In [189]:
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 24
prediction_matrix_lstm_r24 = produce_h_step_forecasting_errors(scaled_rv_gjrgarch, pred_matrix_gjrgarch_24, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift, benchmark_vol)

313/313 [==============================] - 27s 85ms/step
MSE: LSTM & hybrid models 24-step ahead forecast
0.001124256512012846
0.0008524767024243834
0.0009547752934914336
0.0008045731285482227
0.0010853780901918149
0.0008764541580185431
MAE: LSTM & hybrid models 24-step ahead forecast
0.021373747030777726
0.015290843589405188
0.01727883203667553
0.01337805411039366
0.021522221272234136
0.01598249464180834
MSE & MAE: realized volatility 24-step ahead forecast
0.0015596194157176276
0.019901079862948184


# 4.5 MS-GARCH data

In [190]:
# Calculate realized volatility based on simulated log-returns
p = 50
rv_msgarch = rv_calc(sim_msgarch_draws, p)

# Realized volatility data
scaled_rv_msgarch = scaled_data(rv_msgarch)
scaled_rv_msgarch = scaled_rv_msgarch.flatten()

# Conditional volatility benchmark
benchmark_vol = sim_msgarch_condvol

## 4.5.1 1-step ahead forecasts

In [191]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 1
prediction_matrix_lstm_m1 = produce_h_step_forecasting_errors(scaled_rv_msgarch, pred_matrix_msgarch_1, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift, benchmark_vol)

313/313 [==============================] - 8s 25ms/step
MSE: LSTM & hybrid models 1-step ahead forecast
0.0004394842514381726
0.0004119409309997381
0.00041516141379166656
0.00039970932490167
0.00044723253337095834
0.00045367139108778184
MAE: LSTM & hybrid models 1-step ahead forecast
0.014099242398702647
0.013145249223892383
0.013370419377347939
0.013310340240928411
0.014287815031789866
0.01441643763693671
MSE & MAE: realized volatility 1-step ahead forecast
0.0004273050034877354
0.013866317269716174


## 4.5.2 12-step ahead forecasts

In [192]:
obs_test = 20000
start = 10000
d = 100
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 12-step ahead forecasts using LSTM
h = 12
prediction_matrix_lstm_m12 = produce_h_step_forecasting_errors(scaled_rv_msgarch, pred_matrix_msgarch_12, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift, benchmark_vol)

313/313 [==============================] - 13s 40ms/step
MSE: LSTM & hybrid models 12-step ahead forecast
0.00044234232518376997
0.0004252648812223827
0.0004043958005976944
0.000396458622948111
0.0003721526713369417
0.00042652817679615586
MAE: LSTM & hybrid models 12-step ahead forecast
0.013566539053179753
0.012944674501074945
0.012525105614245547
0.012363149509267705
0.012290901098092912
0.012994452503302681
MSE & MAE: realized volatility 12-step ahead forecast
0.0004684172768278799
0.01404873139773857


## 4.5.3 24-step ahead forecasts

In [203]:
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

# Producing 1-step ahead forecasts using LSTM
h = 24
prediction_matrix_lstm_m24 = produce_h_step_forecasting_errors(scaled_rv_msgarch, pred_matrix_msgarch_24, obs_test, start, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift, benchmark_vol)

313/313 [==============================] - 27s 83ms/step
MSE: LSTM & hybrid models 24-step ahead forecast
0.00026328760808492515
0.0002449895711335309
0.000257866191043404
0.000263369741961411
0.0002461222096886691
0.00024379766549017363
MAE: LSTM & hybrid models 24-step ahead forecast
0.009822702376902961
0.008865368581689279
0.008652707218055326
0.01009547494179272
0.008087861562661426
0.007988702637095979
MSE & MAE: realized volatility 24-step ahead forecast
0.00029154263013214356
0.008818468020810735


# Test all predictions with plots

In [33]:
plt.plot(prediction_matrix_lstm_m1[:,0])
plt.plot(prediction_matrix_lstm_m12[:,0])
plt.plot(prediction_matrix_lstm_m24[:,0])

NameError: name 'prediction_matrix_lstm_m1' is not defined

In [202]:
# Function h-step ahead

def produce_h_step_forecasting_errors(scaled_rv, pred_matrix, obs_test, d, p, h, epochs, units, nodes, verbose, LR, activation_lstm, rv, shift, benchmark_vol):
    
    # Unpack prediction matrix
    scaled_pred_arch = scaled_data(pred_matrix[:, 0]).flatten()
    scaled_pred_garch = scaled_data(pred_matrix[:, 1]).flatten()
    scaled_pred_egarch = scaled_data(pred_matrix[:, 2]).flatten()
    scaled_pred_gjrgarch = scaled_data(pred_matrix[:, 3]).flatten()
    
    if np.shape(pred_matrix)[1]==5:
        scaled_pred_msgarch = scaled_data(pred_matrix[:, 4]).flatten()

    # Generate predictions (p = window size realized volatility) (Might remove (p-1)!)
    lstm_preds = univariate_lstm_preds_shift(scaled_rv, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv, shift)
    lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv, scaled_pred_arch[(p-1):], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv, h, shift)
    lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv, scaled_pred_garch[(p-1):], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv, h, shift)
    lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv, scaled_pred_egarch[(p-1):], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv, h, shift)
    lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv, scaled_pred_gjrgarch[(p-1):], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv, h, shift)
    lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv, scaled_pred_msgarch[(p-1):], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv, h, shift)

    # Execute if h>1: Retrieving every h-th sequence, for h-th step ahead prediction evaluation
    if h!=1:        
        lstm_preds = create_h_step_forecasts(lstm_preds, h, obs_test)
        lstm_preds_arch = create_h_step_forecasts(lstm_preds_arch, h, obs_test)
        lstm_preds_garch = create_h_step_forecasts(lstm_preds_garch, h, obs_test)
        lstm_preds_egarch = create_h_step_forecasts(lstm_preds_egarch, h, obs_test)
        lstm_preds_gjrgarch = create_h_step_forecasts(lstm_preds_gjrgarch, h, obs_test)
        lstm_preds_msgarch = create_h_step_forecasts(lstm_preds_msgarch, h, obs_test)

    # Error calculations (MSE)
    print(f"MSE: LSTM & hybrid models {h}-step ahead forecast")
    print(mse(lstm_preds[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))])) # -(h+1) to ensure same length outputs
    print(mse(lstm_preds_arch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mse(lstm_preds_garch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mse(lstm_preds_egarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mse(lstm_preds_gjrgarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mse(lstm_preds_msgarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))

    # Error calculations (MAE)
    print(f"MAE: LSTM & hybrid models {h}-step ahead forecast")
    print(mae(lstm_preds[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))])) # -(h+1) to ensure same length outputs
    print(mae(lstm_preds_arch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mae(lstm_preds_garch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mae(lstm_preds_egarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mae(lstm_preds_gjrgarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))
    print(mae(lstm_preds_msgarch[:obs_test-(h+1)], benchmark_vol[(len(benchmark_vol) - obs_test):(len(benchmark_vol)-(h+1))]))

    # Realized variance errors (MSE & MAE)
    print(f"MSE & MAE: realized volatility {h}-step ahead forecast")
    print(mse(rv[(len(rv) - obs_test - h):(len(rv) - h)], benchmark_vol[(len(benchmark_vol)-obs_test):len(benchmark_vol)]))
    print(mae(rv[(len(rv) - obs_test - h):(len(rv) - h)], benchmark_vol[(len(benchmark_vol)-obs_test):len(benchmark_vol)]))
    
    # Flatten predictions to put them into 1 matrix
    lstm_preds_arch = lstm_preds_arch.flatten()
    lstm_preds_garch = lstm_preds_garch.flatten()
    lstm_preds_egarch = lstm_preds_egarch.flatten()
    lstm_preds_gjrgarch = lstm_preds_gjrgarch.flatten()
    lstm_preds_msgarch = lstm_preds_msgarch.flatten()
    
    prediction_matrix_lstm = np.vstack((lstm_preds, lstm_preds_arch, lstm_preds_garch, lstm_preds_egarch, lstm_preds_gjrgarch, lstm_preds_msgarch))
    
    return prediction_matrix_lstm

## Without functions 

In [18]:
arch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_arch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift)
arch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm,rv_arch, h, shift)

313/313 [==============================] - 4s 12ms/step


In [19]:
# Error calculations
print('MSE: LSTM & hybrid models (1 step ahead)')
print(mse(arch_lstm_preds, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_arch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_garch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_egarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_gjrgarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_msgarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Error calculations
print('MAE: LSTM & hybrid models (1 step ahead)')
print(mae(arch_lstm_preds, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_arch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_garch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_egarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_gjrgarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_msgarch, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (1 step ahead)')
print(mse(rv_arch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_arch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models
0.0002312610690765773
0.00023489433166557278
0.0002306550269078805
0.0002323863690279035
0.00022784876781001585
0.00022852642132672066
MAE: LSTM & hybrid models
0.007763846140148484
0.007827603711220375
0.007718975930620201
0.0077279989426974225
0.0076028710590690055
0.00767753812635609
MSE & MAE: realized volatility forecasts
0.00023362855856521016
0.007807885435692457


## 4.1.2 ARCH sequence: 12-step ahead forecasting

In [20]:
obs_test = 20000
start = 10000
d = 100
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [21]:
h = 12

In [22]:
# Calculating prediction matrices
arch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_arch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift)
arch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
arch_lstm_preds_h = create_h_step_forecasts(arch_lstm_preds, h, obs_test)
arch_lstm_preds_arch_h = create_h_step_forecasts(arch_lstm_preds_arch, h, obs_test)
arch_lstm_preds_garch_h = create_h_step_forecasts(arch_lstm_preds_garch, h, obs_test)
arch_lstm_preds_egarch_h = create_h_step_forecasts(arch_lstm_preds_egarch, h, obs_test)
arch_lstm_preds_gjrgarch_h = create_h_step_forecasts(arch_lstm_preds_gjrgarch, h, obs_test)
arch_lstm_preds_msgarch_h = create_h_step_forecasts(arch_lstm_preds_msgarch, h, obs_test)

313/313 [==============================] - 7s 23ms/step


In [24]:
# Error calculations
print('MSE: LSTM & hybrid models (12 step ahead)')
print(mse(arch_lstm_preds_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_arch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_garch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_egarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_gjrgarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(arch_lstm_preds_msgarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Error calculations
print('MAE: LSTM & hybrid models (12 step ahead)')
print(mae(arch_lstm_preds_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_arch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_garch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_egarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_gjrgarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(arch_lstm_preds_msgarch_h[:-4], sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (12 step ahead)')
print(mse(rv_arch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_arch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (12 step ahead)
0.0002604288680254772
0.0002588069289715121
0.00026638696461895824
0.0002495256314017465
0.0002613394340692686
0.00024723957486378647
MAE: LSTM & hybrid models (12 step ahead)
0.007905820185271754
0.007906815782157392
0.008009426338845608
0.007330936525021002
0.0077629151840993135
0.007199769634746777
MSE & MAE: realized volatility forecasts (12 step ahead)
0.0002801909544288314
0.008297771760459234


## 4.1.3 ARCH sequence: 24-step ahead forecasting

In [53]:
obs_test = 20000
start = 10000
d = 200
epochs = 75
units = 50
nodes = 75
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [54]:
h = 24

In [55]:
# Calculating prediction matrices
arch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_arch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_arch, shift)
arch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)
arch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_arch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_arch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
arch_lstm_preds_h = create_h_step_forecasts(arch_lstm_preds, h, obs_test)
arch_lstm_preds_arch_h = create_h_step_forecasts(arch_lstm_preds_arch, h, obs_test)
arch_lstm_preds_garch_h = create_h_step_forecasts(arch_lstm_preds_garch, h, obs_test)
arch_lstm_preds_egarch_h = create_h_step_forecasts(arch_lstm_preds_egarch, h, obs_test)
arch_lstm_preds_gjrgarch_h = create_h_step_forecasts(arch_lstm_preds_gjrgarch, h, obs_test)
arch_lstm_preds_msgarch_h = create_h_step_forecasts(arch_lstm_preds_msgarch, h, obs_test)

313/313 [==============================] - 5s 16ms/step


In [56]:
# Error calculations
print('MSE: LSTM & hybrid models (24 step ahead)')
print(mse(arch_lstm_preds_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mse(arch_lstm_preds_arch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mse(arch_lstm_preds_garch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mse(arch_lstm_preds_egarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mse(arch_lstm_preds_gjrgarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mse(arch_lstm_preds_msgarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))

# Error calculations
print('MAE: LSTM & hybrid models (24 step ahead)')
print(mae(arch_lstm_preds_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mae(arch_lstm_preds_arch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mae(arch_lstm_preds_garch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mae(arch_lstm_preds_egarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mae(arch_lstm_preds_gjrgarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))
print(mae(arch_lstm_preds_msgarch_h, sim_arch_condvol[(len(sim_msgarch_condvol) - obs_test):(len(sim_msgarch_condvol)-8)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (24 step ahead)')
print(mse(rv_arch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_arch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_arch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (24 step ahead)


ValueError: Found input variables with inconsistent numbers of samples: [19992, 20000]

## 4.2 GARCH data

In [28]:
scaled_rv_garch = scaled_data(rv_garch)
scaled_filt_arch = scaled_data(garch_filt_arch.to_numpy())
scaled_filt_garch = scaled_data(garch_filt_garch.to_numpy())
scaled_filt_egarch = scaled_data(garch_filt_egarch.to_numpy())
scaled_filt_gjrgarch = scaled_data(garch_filt_gjrgarch.to_numpy())
scaled_filt_msgarch = scaled_data(garch_filt_msgarch.to_numpy())

In [29]:
scaled_rv_garch = scaled_rv_garch.flatten()
scaled_filt_arch = scaled_filt_arch.flatten()
scaled_filt_garch = scaled_filt_garch.flatten()
scaled_filt_egarch = scaled_filt_egarch.flatten()
scaled_filt_gjrgarch = scaled_filt_gjrgarch.flatten()
scaled_filt_msgarch = scaled_filt_msgarch.flatten()

## 4.2.1 GARCH sequence: 1-step ahead forecasting

In [30]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [31]:
h = 1

In [32]:
garch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_garch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift)
garch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)

313/313 [==============================] - 6s 18ms/step


In [33]:
# Error calculations
print('MSE: LSTM & hybrid models (1 step ahead)')
print(mse(garch_lstm_preds, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_arch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_garch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_egarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_gjrgarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_msgarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Error calculations
print('MAE: LSTM & hybrid models (1 step ahead)')
print(mae(garch_lstm_preds, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_arch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_garch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_egarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_gjrgarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_msgarch, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (1 step ahead)')
print(mse(rv_garch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_garch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (1 step ahead)
0.00019205947525916534
0.00021014988855959675
0.00019560534452855925
0.00019681913165983295
0.00018646709694609558
0.00018797397038312395
MAE: LSTM & hybrid models (1 step ahead)
0.008884041020233916
0.009959655535865133
0.009402962981749788
0.008846167668665937
0.008586938222599571
0.008894320514563852
MSE & MAE: realized volatility forecasts (1 step ahead)
0.00020869793429565168
0.009259902988863401


## 4.2.2 GARCH sequence: 12-step ahead forecasting

In [57]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [58]:
h = 12

In [59]:
# Calculating prediction matrices
garch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_garch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift)
garch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
garch_lstm_preds_h = create_h_step_forecasts(garch_lstm_preds, h, obs_test)
garch_lstm_preds_arch_h = create_h_step_forecasts(garch_lstm_preds_arch, h, obs_test)
garch_lstm_preds_garch_h = create_h_step_forecasts(garch_lstm_preds_garch, h, obs_test)
garch_lstm_preds_egarch_h = create_h_step_forecasts(garch_lstm_preds_egarch, h, obs_test)
garch_lstm_preds_gjrgarch_h = create_h_step_forecasts(garch_lstm_preds_gjrgarch, h, obs_test)
garch_lstm_preds_msgarch_h = create_h_step_forecasts(garch_lstm_preds_msgarch, h, obs_test)

313/313 [==============================] - 4s 13ms/step


In [60]:
# Error calculations
print('MSE: LSTM & hybrid models (12-step ahead)')
print(mse(garch_lstm_preds_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_arch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_garch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_egarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_gjrgarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_msgarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Error calculations
print('MAE: LSTM & hybrid models (12-step ahead)')
print(mae(garch_lstm_preds_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_arch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_garch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_egarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_gjrgarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_msgarch_h[:-4], sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (12-step ahead)')
print(mse(rv_garch[(len(rv_garch)-obs_test-12):(len(rv_garch)-1)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_garch[(len(rv_garch)-obs_test-12):(len(rv_garch)-1)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (12-step ahead)


ValueError: Found input variables with inconsistent numbers of samples: [20004, 20000]

## 4.2.3 GARCH sequence: 24-step ahead forecasting

In [61]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [62]:
h = 24

In [63]:
# Calculating prediction matrices
garch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_garch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_garch, shift)
garch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)
garch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_garch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_garch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
garch_lstm_preds_h = create_h_step_forecasts(garch_lstm_preds, h, obs_test)
garch_lstm_preds_arch_h = create_h_step_forecasts(garch_lstm_preds_arch, h, obs_test)
garch_lstm_preds_garch_h = create_h_step_forecasts(garch_lstm_preds_garch, h, obs_test)
garch_lstm_preds_egarch_h = create_h_step_forecasts(garch_lstm_preds_egarch, h, obs_test)
garch_lstm_preds_gjrgarch_h = create_h_step_forecasts(garch_lstm_preds_gjrgarch, h, obs_test)
garch_lstm_preds_msgarch_h = create_h_step_forecasts(garch_lstm_preds_msgarch, h, obs_test)

313/313 [==============================] - 7s 22ms/step


In [64]:
# Error calculations
print('MSE: LSTM & hybrid models (24 step ahead)')
print(mse(garch_lstm_preds_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_arch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_garch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_egarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_gjrgarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mse(garch_lstm_preds_msgarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Error calculations
print('MAE: LSTM & hybrid models (24 step ahead)')
print(mae(garch_lstm_preds_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_arch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_garch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_egarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_gjrgarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))
print(mae(garch_lstm_preds_msgarch_h, sim_garch_condvol[(len(sim_msgarch_condvol) - obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (24 step ahead)')
print(mse(rv_garch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_garch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_garch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (24 step ahead)


ValueError: Found input variables with inconsistent numbers of samples: [19992, 20000]

## 4.3 E-GARCH data

In [38]:
scaled_rv_egarch = scaled_data(rv_egarch)
scaled_filt_arch = scaled_data(egarch_filt_arch.to_numpy())
scaled_filt_garch = scaled_data(egarch_filt_garch.to_numpy())
scaled_filt_egarch = scaled_data(egarch_filt_egarch.to_numpy())
scaled_filt_gjrgarch = scaled_data(egarch_filt_gjrgarch.to_numpy())
scaled_filt_msgarch = scaled_data(egarch_filt_msgarch.to_numpy())

scaled_rv_egarch = scaled_rv_egarch.flatten()
scaled_filt_arch = scaled_filt_arch.flatten()
scaled_filt_garch = scaled_filt_garch.flatten()
scaled_filt_egarch = scaled_filt_egarch.flatten()
scaled_filt_gjrgarch = scaled_filt_gjrgarch.flatten()
scaled_filt_msgarch = scaled_filt_msgarch.flatten()

### 4.3.1 1-step ahead forecasting

In [39]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [40]:
h = 1

In [41]:
egarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_egarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift)
egarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)

313/313 [==============================] - 6s 17ms/step


In [42]:
# Mean squared errors
print('MSE: LSTM & hybrid models (1 step ahead)')
print(mse(egarch_lstm_preds, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_arch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_garch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_egarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_gjrgarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_msgarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (1 step ahead)')
print(mae(egarch_lstm_preds, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_arch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_garch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_egarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_gjrgarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_msgarch, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (1 step ahead)')
print(mse(rv_egarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_egarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (1 step ahead)
0.013389701036022222
0.013362820148230332
0.0131719960735521
0.014204588718060757
0.014564833523429827
0.013523167462505382
MAE: LSTM & hybrid models (1 step ahead)
0.09282088239302957
0.09221411601321436
0.09180289157974036
0.09565127532028919
0.09681685336505336
0.09350830118082795
MSE & MAE: realized volatility forecasts (1 step ahead)
0.014417210637362452
0.09658524957276071


### 4.3.2 12-step ahead forecasting

In [65]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [66]:
h = 12 

In [67]:
# Generating prediction matrices
egarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_egarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift)
egarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
egarch_lstm_preds_h = create_h_step_forecasts(egarch_lstm_preds, h, obs_test)
egarch_lstm_preds_arch_h = create_h_step_forecasts(egarch_lstm_preds_arch, h, obs_test)
egarch_lstm_preds_garch_h = create_h_step_forecasts(egarch_lstm_preds_garch, h, obs_test)
egarch_lstm_preds_egarch_h = create_h_step_forecasts(egarch_lstm_preds_egarch, h, obs_test)
egarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(egarch_lstm_preds_gjrgarch, h, obs_test)
egarch_lstm_preds_msgarch_h = create_h_step_forecasts(egarch_lstm_preds_msgarch, h, obs_test)

313/313 [==============================] - 10s 28ms/step


In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (12-step ahead)')
print(mse(egarch_lstm_preds_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_arch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_garch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_egarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_gjrgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_msgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (12-step ahead)')
print(mae(egarch_lstm_preds_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_arch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_garch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_egarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_gjrgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_msgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (12 step ahead)')
print(mse(rv_egarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_egarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

### 4.3.3 24-step ahead forecasting

In [ ]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [ ]:
h = 24

In [ ]:
# Generating prediction matrices
egarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_egarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_egarch, shift)
egarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)
egarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_egarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_egarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
egarch_lstm_preds_h = create_h_step_forecasts(egarch_lstm_preds, h, obs_test)
egarch_lstm_preds_arch_h = create_h_step_forecasts(egarch_lstm_preds_arch, h, obs_test)
egarch_lstm_preds_garch_h = create_h_step_forecasts(egarch_lstm_preds_garch, h, obs_test)
egarch_lstm_preds_egarch_h = create_h_step_forecasts(egarch_lstm_preds_egarch, h, obs_test)
egarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(egarch_lstm_preds_gjrgarch, h, obs_test)
egarch_lstm_preds_msgarch_h = create_h_step_forecasts(egarch_lstm_preds_msgarch, h, obs_test)

In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (24-step ahead)')
print(mse(egarch_lstm_preds_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_arch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_garch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_egarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_gjrgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(egarch_lstm_preds_msgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (24-step ahead)')
print(mae(egarch_lstm_preds_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_arch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_garch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_egarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_gjrgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(egarch_lstm_preds_msgarch_h, sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: Realized volatility forecasts (24-step ahead)')
print(mse(rv_egarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_egarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-1)],sim_egarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

## 4.4 GJR-GARCH data

In [43]:
scaled_rv_gjrgarch = scaled_data(rv_gjrgarch)
scaled_filt_arch = scaled_data(gjrgarch_filt_arch.to_numpy())
scaled_filt_garch = scaled_data(gjrgarch_filt_garch.to_numpy())
scaled_filt_egarch = scaled_data(gjrgarch_filt_egarch.to_numpy())
scaled_filt_gjrgarch = scaled_data(gjrgarch_filt_gjrgarch.to_numpy())
scaled_filt_msgarch = scaled_data(gjrgarch_filt_msgarch.to_numpy())

scaled_rv_gjrgarch = scaled_rv_gjrgarch.flatten()
scaled_filt_arch = scaled_filt_arch.flatten()
scaled_filt_garch = scaled_filt_garch.flatten()
scaled_filt_egarch = scaled_filt_egarch.flatten()
scaled_filt_gjrgarch = scaled_filt_gjrgarch.flatten()
scaled_filt_msgarch = scaled_filt_msgarch.flatten()

### 4.4.1 1-step ahead forecasting

In [44]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [45]:
h = 1

In [46]:
gjrgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_gjrgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift)
gjrgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)

313/313 [==============================] - 6s 19ms/step


In [47]:
# Mean squared errors
print('MSE: LSTM & hybrid models (1-step ahead)')
print(mse(gjrgarch_lstm_preds, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_arch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_garch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_egarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_gjrgarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_msgarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (1-step ahead)')
print(mae(gjrgarch_lstm_preds, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_arch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_garch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_egarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_gjrgarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_msgarch, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: Realized volatility forecasts (1-step ahead)')
print(mse(rv_gjrgarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_gjrgarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (1-step ahead)
0.0008972834372368752
0.0007429353197907071
0.0007775943788899068
0.0010684994099369655
0.001200542567654619
0.0006954749565160367
MAE: LSTM & hybrid models (1-step ahead)
0.0177420889793409
0.015000396941040009
0.015639167281722093
0.021047210460492072
0.024146381328412684
0.01384127290966845
MSE & MAE: Realized volatility forecasts (1-step ahead)
0.0007906618097120884
0.014420564284997571


### 4.4.2 12-step ahead forecasting

In [ ]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [ ]:
h = 12

In [ ]:
gjrgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_gjrgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift)
gjrgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
gjrgarch_lstm_preds_h = create_h_step_forecasts(gjrgarch_lstm_preds, h, obs_test)
gjrgarch_lstm_preds_arch_h = create_h_step_forecasts(gjrgarch_lstm_preds_arch, h, obs_test)
gjrgarch_lstm_preds_garch_h = create_h_step_forecasts(gjrgarch_lstm_preds_garch, h, obs_test)
gjrgarch_lstm_preds_egarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_egarch, h, obs_test)
gjrgarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_gjrgarch, h, obs_test)
gjrgarch_lstm_preds_msgarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_msgarch, h, obs_test)

In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (12-step ahead)')
print(mse(gjrgarch_lstm_preds_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_arch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_garch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_egarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_gjrgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_msgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (12-step ahead)')
print(mae(gjrgarch_lstm_preds_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_arch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_garch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_egarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_gjrgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_msgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: Realized volatility forecasts (12-step ahead)')
print(mse(rv_gjrgarch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_gjrgarch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

### 4.4.3 24-step ahead forecasting

In [ ]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [ ]:
h = 24

In [ ]:
gjrgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_gjrgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_gjrgarch, shift)
gjrgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)
gjrgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_gjrgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_gjrgarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
gjrgarch_lstm_preds_h = create_h_step_forecasts(gjrgarch_lstm_preds, h, obs_test)
gjrgarch_lstm_preds_arch_h = create_h_step_forecasts(gjrgarch_lstm_preds_arch, h, obs_test)
gjrgarch_lstm_preds_garch_h = create_h_step_forecasts(gjrgarch_lstm_preds_garch, h, obs_test)
gjrgarch_lstm_preds_egarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_egarch, h, obs_test)
gjrgarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_gjrgarch, h, obs_test)
gjrgarch_lstm_preds_msgarch_h = create_h_step_forecasts(gjrgarch_lstm_preds_msgarch, h, obs_test)

In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (24-step ahead)')
print(mse(gjrgarch_lstm_preds_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_arch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_garch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_egarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_gjrgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(gjrgarch_lstm_preds_msgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (24-step ahead)')
print(mae(gjrgarch_lstm_preds_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_arch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_garch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_egarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_gjrgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(gjrgarch_lstm_preds_msgarch_h, sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: Realized volatility forecasts (24-step ahead)')
print(mse(rv_gjrgarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_gjrgarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_gjrgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

## 4.5 MS-GARCH data

In [48]:
scaled_rv_msgarch = scaled_data(rv_msgarch)
scaled_filt_arch = scaled_data(msgarch_filt_arch.to_numpy())
scaled_filt_garch = scaled_data(msgarch_filt_garch.to_numpy())
scaled_filt_egarch = scaled_data(msgarch_filt_egarch.to_numpy())
scaled_filt_gjrgarch = scaled_data(msgarch_filt_gjrgarch.to_numpy())
scaled_filt_msgarch = scaled_data(msgarch_filt_msgarch.to_numpy())

scaled_rv_msgarch = scaled_rv_msgarch.flatten()
scaled_filt_arch = scaled_filt_arch.flatten()
scaled_filt_garch = scaled_filt_garch.flatten()
scaled_filt_egarch = scaled_filt_egarch.flatten()
scaled_filt_gjrgarch = scaled_filt_gjrgarch.flatten()
scaled_filt_msgarch = scaled_filt_msgarch.flatten()

### 4.5.1 1-step ahead forecasting

In [49]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [50]:
h = 1

In [51]:
msgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_msgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift)
msgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)

313/313 [==============================] - 6s 19ms/step


In [52]:
# Mean squared errors
print('MSE: LSTM & hybrid models (1-step ahead)')
print(mse(msgarch_lstm_preds, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_arch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_garch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_egarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_gjrgarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_msgarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (1-step ahead)')
print(mae(msgarch_lstm_preds, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_arch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_garch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_egarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_gjrgarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_msgarch, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (1-step ahead)')
print(mse(rv_msgarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_msgarch[(len(rv_garch)-obs_test-1):(len(rv_garch)-1)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

MSE: LSTM & hybrid models (1-step ahead)
0.00043079150630620164
0.0004252450741757477
0.0003970336740763269
0.0004242055232417698
0.0004393809206843335
0.0004315128231583512
MAE: LSTM & hybrid models (1-step ahead)
0.013838104636866737
0.013571247609489603
0.012951037173736054
0.013650043396508025
0.014086500401495389
0.013788031755036722
MSE & MAE: realized volatility forecasts (1-step ahead)
0.0004273050034877354
0.013866317269716174


### 4.5.2 12-step ahead forecasting

In [ ]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [ ]:
h = 12

In [ ]:
msgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_msgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift)
msgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
msgarch_lstm_preds_h = create_h_step_forecasts(msgarch_lstm_preds, h, obs_test)
msgarch_lstm_preds_arch_h = create_h_step_forecasts(msgarch_lstm_preds_arch, h, obs_test)
msgarch_lstm_preds_garch_h = create_h_step_forecasts(msgarch_lstm_preds_garch, h, obs_test)
msgarch_lstm_preds_egarch_h = create_h_step_forecasts(msgarch_lstm_preds_egarch, h, obs_test)
msgarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(msgarch_lstm_preds_gjrgarch, h, obs_test)
msgarch_lstm_preds_msgarch_h = create_h_step_forecasts(msgarch_lstm_preds_msgarch, h, obs_test)

In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (12-step ahead)')
print(mse(msgarch_lstm_preds_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_arch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_garch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_egarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_gjrgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_msgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (12-step ahead)')
print(mae(msgarch_lstm_preds_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_arch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_garch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_egarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_gjrgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_msgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (12-step ahead)')
print(mse(rv_msgarch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_msgarch[(len(rv_garch)-obs_test-12):(len(rv_garch)-12)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

### 4.5.3 24-step ahead forecasting

In [ ]:
obs_test = 20000
start = 10000
d = 50
epochs = 75
units = 100
nodes = 150
verbose = 0
LR = 0.0001
activation_lstm = 'tanh'
shift = 25

In [ ]:
h = 24

In [ ]:
msgarch_lstm_preds = univariate_lstm_preds_shift(scaled_rv_msgarch, obs_test, d, h, epochs, units, nodes, verbose, LR, activation_lstm, rv_msgarch, shift)
msgarch_lstm_preds_arch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_arch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_garch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_garch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_egarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_egarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_gjrgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_gjrgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)
msgarch_lstm_preds_msgarch = generate_2_forecasts_hybrid_shift(scaled_rv_msgarch, scaled_filt_msgarch[49:], obs_test, start, d, epochs, units, nodes, LR, activation_lstm, rv_msgarch, h, shift)

# Retrieving every h-th sequence, for h-th step ahead prediction evaluation
msgarch_lstm_preds_h = create_h_step_forecasts(msgarch_lstm_preds, h, obs_test)
msgarch_lstm_preds_arch_h = create_h_step_forecasts(msgarch_lstm_preds_arch, h, obs_test)
msgarch_lstm_preds_garch_h = create_h_step_forecasts(msgarch_lstm_preds_garch, h, obs_test)
msgarch_lstm_preds_egarch_h = create_h_step_forecasts(msgarch_lstm_preds_egarch, h, obs_test)
msgarch_lstm_preds_gjrgarch_h = create_h_step_forecasts(msgarch_lstm_preds_gjrgarch, h, obs_test)
msgarch_lstm_preds_msgarch_h = create_h_step_forecasts(msgarch_lstm_preds_msgarch, h, obs_test)

In [ ]:
# Mean squared errors
print('MSE: LSTM & hybrid models (24-step ahead)')
print(mse(msgarch_lstm_preds_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_arch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_garch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_egarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_gjrgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mse(msgarch_lstm_preds_msgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Mean absolute errors
print('MAE: LSTM & hybrid models (24-step ahead)')
print(mae(msgarch_lstm_preds_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_arch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_garch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_egarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_gjrgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(msgarch_lstm_preds_msgarch_h, sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))

# Realized variance errors
print('MSE & MAE: realized volatility forecasts (24-step ahead)')
print(mse(rv_msgarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))
print(mae(rv_msgarch[(len(rv_garch)-obs_test-24):(len(rv_garch)-24)],sim_msgarch_condvol[(len(sim_msgarch_condvol)-obs_test):len(sim_msgarch_condvol)]))